In [2]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda', index=0)

In [11]:
from torch.utils.data import Dataset
import json
import cv2
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn


In [7]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [18]:
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = True)

In [19]:
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

# Load ResNet-50 with pre-trained weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)

# Replace the final classification layer with keypoint regression layer
model.fc = nn.Linear(model.fc.in_features, 14 * 2)


In [20]:
model = model.to(device)

In [21]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [22]:
epochs=50
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14488.716796875
Epoch 0, iter 10, loss: 14559.8291015625
Epoch 0, iter 20, loss: 14559.5498046875
Epoch 0, iter 30, loss: 14164.0341796875
Epoch 0, iter 40, loss: 12689.228515625
Epoch 0, iter 50, loss: 11866.1318359375
Epoch 0, iter 60, loss: 10831.966796875
Epoch 0, iter 70, loss: 9436.294921875
Epoch 0, iter 80, loss: 8230.4375
Epoch 0, iter 90, loss: 8205.3544921875
Epoch 0, iter 100, loss: 7322.865234375
Epoch 0, iter 110, loss: 6954.8017578125
Epoch 0, iter 120, loss: 6147.87841796875
Epoch 0, iter 130, loss: 5407.40234375
Epoch 0, iter 140, loss: 5461.4658203125
Epoch 0, iter 150, loss: 4526.49365234375
Epoch 0, iter 160, loss: 4129.54296875
Epoch 0, iter 170, loss: 3764.36572265625
Epoch 0, iter 180, loss: 3237.75537109375
Epoch 0, iter 190, loss: 3082.52490234375
Epoch 0, iter 200, loss: 2884.141845703125
Epoch 1, iter 0, loss: 2529.12060546875
Epoch 1, iter 10, loss: 2523.74169921875
Epoch 1, iter 20, loss: 2078.47705078125
Epoch 1, iter 30, loss: 1952.

In [23]:
torch.save(model.state_dict(), "keypoints_model2_50.pth")

In [24]:
import os
print(os.getcwd())

C:\Users\srina\OneDrive\Desktop\nsksk\tennis-analysis\notebooks
